# Практическая работа №6. Атака по переносу (Transfer Attack) на модели ИИ
## Выполнил студент группы ББМО-01-23 Агасиев Максим Артурович

### 1. Загрузка и создание двух различных моделей


In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])

# Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

# Обучение модели
model1.fit(train_images, train_labels, epochs=5)

# Сохранение модели
model1.save('mnist_model1.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])

# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

# Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)

# Сохранение модели
model2.save('mnist_model2.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8781 - loss: 0.4306
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9637 - loss: 0.1253
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9745 - loss: 0.0832
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9821 - loss: 0.0584
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9869 - loss: 0.0436


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9130 - loss: 0.2936
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9846 - loss: 0.0505
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9911 - loss: 0.0297
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9938 - loss: 0.0185
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9960 - loss: 0.0121


### 2. Реализация атаки FGSM на первую модель

In [2]:
import numpy as np

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  # Применение знака градиента к изображению
  perturbed_image = image + epsilon * np.sign(gradient)
  perturbed_image = np.clip(perturbed_image, 0, 1)
  return perturbed_image

# Генерация противоречивых примеров для первой модели
def generate_adversarial_example(model, image, label, epsilon):
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))

    # Если label — это one-hot вектор, то преобразуем его в индекс
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)

    # Вычисление градиента
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)

    gradient = tape.gradient(loss, image)

    # Применяем FGSM
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())

    return np.reshape(adversarial_image, (28, 28, 1))

def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))

    adversarial_images = np.array(adversarial_images)

    print("Shape of adversarial_images:", adversarial_images.shape)

    return adversarial_images


# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)

Shape of adversarial_images: (10000, 28, 28)


### 3. Оценка противоречивых примеров на обеих моделях

In [3]:
# Оценка первой модели на противоречивых примерах
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1087 - loss: 6.8504
Accuracy of model1 on adversarial examples: 0.1307000070810318
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9581 - loss: 0.1451
Accuracy of model2 on adversarial examples from model1: 0.9649999737739563


### 4. Анализ переносимости атак

In [4]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28,
28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9515 - loss: 0.1634
Accuracy of model1 on adversarial examples from model2: 0.9577000141143799


### Вывод

В ходе выполнения работы было исследовано воздействие атаки по переносу, при которой противоречивые примеры, сгенерированные для одной модели, использовались для атаки на другую.

Результаты показали, что первая модель, на которой были сгенерированы противоречивые примеры с использованием FGSM, продемонстрировала резкое падение точности с 95% до 10%, что свидетельствует о высокой уязвимости модели к данной атаке. В свою очередь, вторая модель показала гораздо меньший эффект от атаки FGSM — её точность снизилась незначительно, что указывает на её большую устойчивость к переносу атак, созданных для другой модели.

Результаты демонстрируют, что атака по переносу с использованием FGSM может существенно ослабить точность модели, особенно если примеры сгенерированы для неё самой. В то же время, другая модель, показавшая более высокую устойчивость, остаётся менее подверженной к таким атакам. Эти выводы подчеркивают важность разработки более устойчивых моделей, которые способны сохранять точность при воздействии атак, перенесённых с другой модели.